## Bibliotecas

In [181]:

import os
import json
import random
random.seed(10)

from kaggle.api.kaggle_api_extended import KaggleApi

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.metrics import roc_auc_score, roc_curve

import pickle


## Download data from Titanic

In [29]:

kaggle_name = "leonardokrivickas"
kaggle_apikey = "790e733df487c48659aae54b86fcebc8"

kaggle_path = os.path.join(os.path.expanduser('~'),'.kaggle')
credential_path = os.path.join(kaggle_path,'kaggle.json')


if not os.path.isdir(kaggle_path):
    os.mkdir(kaggle_path)

if not os.path.isfile(credential_path):
    with open(credential_path,'w') as f:
        f.write(json.dumps({"username":kaggle_name,"key":kaggle_apikey}))


api = KaggleApi()
api.authenticate()

if not os.path.isdir('./dataset'):
    os.mkdir('./dataset')

files_to_download = ['gender_submission.csv','test.csv','train.csv']
for file in files_to_download:
    api.competition_download_file('titanic',file,path='./dataset')


gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


## Load train data

In [34]:

train = pd.read_csv('./dataset/train.csv')
train.head(4)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


## Train Base Model

In [317]:

feature = ['Pclass','Sex','SibSp','Parch']

X = train[feature].copy()
y = train['Survived'].copy()

X['Sex'] = X['Sex'].map(lambda x: 1 if x == 'male' else 2)

sss = StratifiedShuffleSplit(n_splits=30, test_size=0.3, random_state=1)

folds_res = list()
for train_idx, test_idx in sss.split(X, y):
    model = RandomForestClassifier(n_estimators=200, max_depth=4, random_state = 1)
    model.fit(X.iloc[train_idx], y.iloc[train_idx])

    predictions = model.predict(X.iloc[test_idx])
    predic_proba = [x[0] for x in model.predict_proba(X.iloc[test_idx])]

    auc = roc_auc_score(y.iloc[test_idx], predictions)
    fpr, tpr, _ = roc_curve(y.iloc[test_idx], predic_proba)
    fpr, tpr = 1-fpr, 1-tpr

    res = {'train_index': train_idx,
            'test_index': test_idx,
            'model': model,
            'fpr': 1-fpr,
            'tpr': 1-tpr,
            'auc': auc}

    folds_res.append(res)

auc_mean = np.mean([fold.get('auc') for fold in folds_res])
print('Mean AUC: {:.4f}'.format(auc_mean))

if not os.path.isdir('./model'):
    os.mkdir('./model')

model_sorted = sorted(folds_res, key=lambda k: k['auc'], reverse=True)

with open('./model/base_model.pkl','wb') as f:
    pickle.dump(model_sorted[0].get('model'), f)


Mean AUC: 0.7723
